# Bsky integration

In [ ]:
//| export

import { BskyXRPC } from "@mary/bluesky-client";
import type {
  AppBskyEmbedExternal,
  AppBskyEmbedImages,
  AppBskyFeedDefs,
  AppBskyFeedPost,
} from "@mary/bluesky-client/lexicons";

type Thread = AppBskyFeedDefs.ThreadViewPost;
type Post = AppBskyFeedDefs.PostView;
type PostRecord = AppBskyFeedPost.Record;
type EmbedImages = AppBskyEmbedImages.View;
type EmbedExternal = AppBskyEmbedExternal.View;

const rpc = new BskyXRPC({ service: "https://public.api.bsky.app" });

# Post URL to @at protocol URI

Figure out how to convert bsky.app URL to a URI we can use to fetch data. We
need to go from this:

```
https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d
```

To smth like this

```
at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
```

`did:plc:ubdeopbbkbgedccgbum7dhsh` is DID for callmephilip.com handle, which
needs to be resolved first

In [ ]:
//| export

const postURLToAtpURI = async (
  postUrl: string,
): Promise<[string, string]> => {
  const urlParts = new URL(postUrl);
  const pathParts = urlParts.pathname.split("/");
  const h = await rpc.get("com.atproto.identity.resolveHandle", {
    params: {
      handle: pathParts[2],
    },
  });

  return [`at://${h.data.did}/app.bsky.feed.post/${pathParts[4]}`, h.data.did];
};

In [ ]:
await postURLToAtpURI(
  "https://bsky.app/profile/jason.energy/post/3ldllxneijk2d",
);

[
  "at://did:plc:ga3wlji66r5mxqch6wh7nq4v/app.bsky.feed.post/3ldllxneijk2d",
  "did:plc:ga3wlji66r5mxqch6wh7nq4v"
]

# Grabbing thread data

Thread has a bunch of nested posts inside replies. Unwrap this into a list of
posts

In [ ]:
//| export

const unwrapThreadPosts = (
  thread: Thread,
): Post[] => {
  const posts: Post[] = [];

  // Add root post
  if (thread.post) {
    posts.push(thread.post);
  }

  // Recursively handle replies
  if (thread.replies) {
    thread.replies.forEach((reply) => {
      posts.push(...unwrapThreadPosts(reply as Thread));
    });
  }

  // Handle nested reply if present
  if ("parent" in thread && thread.parent) {
    posts.push(
      ...unwrapThreadPosts(thread.parent as Thread),
    );
  }

  return posts;
};

type ThreadData = {
  handle: string;
  posts: Post[];
};

export const downloadThread = async (
  postUrl: string,
): Promise<ThreadData> => {
  const [uri, handle] = await postURLToAtpURI(postUrl);
  const d = await rpc.get("app.bsky.feed.getPostThread", {
    params: {
      uri,
    },
  });
  const r = unwrapThreadPosts(d.data.thread as Thread);
  return {
    handle,
    posts: r.sort(
      (a: Post, b: Post) =>
        new Date((a.record as PostRecord).createdAt).getTime() -
        new Date((b.record as PostRecord).createdAt).getTime(),
    ),
  };
};

In [ ]:
const { posts, handle } = await downloadThread(
  // "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
  "https://bsky.app/profile/jason.energy/post/3ldllxneijk2d",
);

# Converting posts to MD

The most basic setup is when there is just a piece of text display. More complex
cases will have links, attachments

In [ ]:
//| export

// from https://github.com/mary-ext/skeetdeck/blob/aa0cb74c0ace489b79d2671c4b9e740ec21623c7/app/api/richtext/unicode.ts

const encoder = new TextEncoder();
const decoder = new TextDecoder();

interface UtfString {
  u16: string;
  u8: Uint8Array;
}

const createUtfString = (utf16: string): UtfString => {
  return {
    u16: utf16,
    u8: encoder.encode(utf16),
  };
};

// const getUtf8Length = (utf: UtfString) => {
// 	return utf.u8.byteLength;
// };

const sliceUtf8 = (utf: UtfString, start?: number, end?: number) => {
  return decoder.decode(utf.u8.slice(start, end));
};

In [ ]:
//| export

const externalEmbedToMarkdown = (
  embed: EmbedExternal,
  authorDid: string,
): string => {
  const { title, description, uri, thumb } = embed.external;

  console.log(embed);

  let thumbUrl: string | undefined;

  if (typeof thumb === "string") {
    thumbUrl = thumb;
  } else if (thumb) {
    thumbUrl =
      // @ts-ignore not sure why this is failing
      `https://cdn.bsky.app/img/feed_thumbnail/plain/${authorDid}/${thumb.ref.$link}@jpeg`;
  }

  const thumbImage = thumbUrl ? `![${title}](${thumbUrl})` : null;

  return [`### [${title}](${uri})`, description, thumbImage || ""]
    .map((l: string) => `> ${l}`)
    .join("\n");
};

// await (async () => {
//   const { posts, handle } = await downloadThread(
//       "https://bsky.app/profile/jason.energy/post/3ldllxneijk2d"
//     );

//   return Deno.jupyter.display(
//     {
//       "text/markdown": externalEmbedToMarkdown(
//         posts[0].record.embed as EmbedExternal,
//         handle
//       ),
//     },
//     { raw: true }
//   );
// })();

In [ ]:
//| export

export const postToMd = (post: Post, handle: string): string => {
  const record = post.record as PostRecord;
  const text = record.text;
  let richtext = text;
  let embeds = "";

  console.log(record);

  if (record.facets) {
    for (const facet of record.facets) {
      for (const feature of facet.features) {
        if (feature.$type === "app.bsky.richtext.facet#link") {
          const linkPlaceholder = sliceUtf8(
            createUtfString(text),
            facet.index.byteStart,
            facet.index.byteEnd,
          );
          richtext = richtext.replace(
            linkPlaceholder,
            `[${linkPlaceholder}](${feature.uri})`,
          );
        } else if (feature.$type === "app.bsky.richtext.facet#mention") {
          const mentionPlaceholder = sliceUtf8(
            createUtfString(text),
            facet.index.byteStart,
            facet.index.byteEnd,
          );
          richtext = richtext.replace(
            mentionPlaceholder,
            `[${mentionPlaceholder}](https://bsky.app/profile/${feature.did})`,
          );
        }
      }
    }
  }

  if (post.embed) {
    if (post.embed.$type === "app.bsky.embed.images#view") {
      const e = post.embed as EmbedImages;
      for (const image of e.images) {
        embeds += `![${
          image.alt || "no image description"
        }](${image.fullsize})\n`;
      }
    } else if (post.embed.$type === "app.bsky.embed.external#view") {
      embeds += externalEmbedToMarkdown(
        post.embed as EmbedExternal,
        handle,
      );
    }
  }

  const [d, t] = record.createdAt.split("T");
  const [h, m] = t.split(":");

  return [
    `> [${post.author.displayName} - @${post.author.handle}](https://bsky.app/profile/${post.author.handle}) **${d} ${
      [h, m].join(
        ":",
      )
    }**`,
    ...richtext.split("\n").filter((l: string) => l.trim() !== ""),
    embeds,
  ].join("\n\n");
};

// await Deno.jupyter.display(
//   {
//     "text/markdown": postToMd(posts[0], handle),
//   },
//   { raw: true }
// );

Support links inside posts

In [ ]:
//| export

export const downloadPostToMd = async (postUrl: string): Promise<string> => {
  const { posts, handle } = await downloadThread(postUrl);

  return posts.map((post) => postToMd(post, handle)).join("\n\n");
};

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": await downloadPostToMd(
      "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
    ),
  },
  { raw: true },
);

{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-08T15:46:20.768Z",
  langs: [ "en" ],
  text: "🧵 Notes on Deno + Jupyter 🦕 🔭"
}


{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-08T15:49:32.248Z",
  facets: [
    {
      features: [
        {
          "$type": "app.bsky.richtext.facet#link",
          uri: "https://docs.deno.com/runtime/reference/cli/jupyter/"
        }
      ],
      index: { byteEnd: 65, byteStart: 36 }
    }
  ],
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "Jupyter kernel for Deno - Deno docs docs.deno.com/runtime/refe..."
}


{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-08T15:56:32.679Z",
  facets: [
    {
      features: [
        {
          "$type": "app.bsky.richtext.facet#link",
          uri: "https://gist.github.com/aaronblondeau/02f94256aab61f409d1f820cc80ea571"
        }
      ],
      index: { byteEnd: 102, byteStart: 71 }
    }
  ],
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreidmwdin3wmdidyziwxr5gk7vxovghdai52dua5mtlcqynbwpfbmve",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcsksuprys2r"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "Repro of original Deno2 jupyter demo with broken mermaid integration - gist.github.com/aaronblondea..."
}


{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-08T16:00:14.858Z",
  embed: {
    "$type": "app.bsky.embed.images",
    images: [
      {
        alt: "",
        aspectRatio: { height: 762, width: 1622 },
        image: {
          "$type": "blob",
          ref: [Object],
          mimeType: "image/jpeg",
          size: 188825
        }
      }
    ]
  },
  facets: [
    {
      features: [
        {
          "$type": "app.bsky.richtext.facet#link",
          uri: "https://deno.land/x/display@v1.1.2"
        }
      ],
      index: { byteEnd: 64, byteStart: 39 }
    }
  ],
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreid2m5inyak7lxot44hn5l572ptm47qsmxbvluqyr2m2ebrflk6ts4",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcsl7foe4s2r"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "Rich dis

{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-08T16:26:17.631Z",
  facets: [
    {
      "$type": "app.bsky.richtext.facet",
      features: [
        {
          "$type": "app.bsky.richtext.facet#mention",
          did: "did:plc:avhotimzhbh7xxwlglyw6c36"
        }
      ],
      index: { byteEnd: 23, byteStart: 0 }
    },
    {
      features: [
        {
          "$type": "app.bsky.richtext.facet#link",
          uri: "https://github.com/rgbkrk/denotebooks"
        }
      ],
      index: { byteEnd: 98, byteStart: 72 }
    }
  ],
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreifivduj2xktczcij36zulvsalmiq2wpk5tzzjjrkji2im4bo7y67y",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcslfzkrr22r"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "@kylekelley.bsky.social has a bunch of Deno in not

...

{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-16T18:03:11.209Z",
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "Hit 2 issues with Deno + Jupyter: running notebooks from cmd and tracking failing tests."
}


{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-16T18:06:10.698Z",
  facets: [
    {
      features: [
        {
          "$type": "app.bsky.richtext.facet#link",
          uri: "https://github.com/jupyter/nbclient/issues/321"
        }
      ],
      index: { byteEnd: 69, byteStart: 43 }
    },
    {
      features: [
        {
          "$type": "app.bsky.richtext.facet#link",
          uri: "https://github.com/callmephilip/jurassic/blob/main/.jurassic/runnb.py"
        }
      ],
      index: { byteEnd: 205, byteStart: 179 }
    }
  ],
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "notebook cmd line issue is described here: git

{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-16T18:07:28.812Z",
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "As far as tests are concerned, ended up dumping them into a separate ts module and then using native `deno test`"
}


{
  "$type": "app.bsky.feed.post",
  createdAt: "2024-12-16T20:24:54.774Z",
  facets: [
    {
      features: [
        {
          "$type": "app.bsky.richtext.facet#link",
          uri: "https://github.com/denoland/deno/pull/20337"
        }
      ],
      index: { byteEnd: 78, byteStart: 52 }
    }
  ],
  langs: [ "en" ],
  reply: {
    parent: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    },
    root: {
      cid: "bafyreidx47j6q5pywcbz23diipd4xdc76xnovct4mi2j5kqbxntdasjarq",
      uri: "at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
    }
  },
  text: "useful reference - PR with Jupyter support for Deno github.com/denoland/den..."
}


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 15:46**

🧵 Notes on Deno + Jupyter 🦕 🔭



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 15:49**

Jupyter kernel for Deno - Deno docs [docs.deno.com/runtime/refe...](https://docs.deno.com/runtime/reference/cli/jupyter/)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 15:56**

Repro of original Deno2 jupyter demo with broken mermaid integration - [gist.github.com/aaronblondea...](https://gist.github.com/aaronblondeau/02f94256aab61f409d1f820cc80ea571)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:00**

Rich displays for jupyter js kernels - [deno.land/x/display@v1...](https://deno.land/x/display@v1.1.2)

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreidfehjdftii7yrynrzeecpqxqjycabduk2yasxeeze5zi3dji4cre@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:26**

[@kylekelley.bsky.social](https://bsky.app/profile/did:plc:avhotimzhbh7xxwlglyw6c36) has a bunch of Deno in notebook examples here - [github.com/rgbkrk/denot...](https://github.com/rgbkrk/denotebooks)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:44**

nice video showing deno + jupyter (via noteable RIP) [www.youtube.com/watch?v=b5OK...](https://www.youtube.com/watch?v=b5OKkMuue3Q)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:53**

anywidget for deno jupyter kernel - [jsr.io/@anywidget/d...](https://jsr.io/@anywidget/deno)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 18:03**

Hit 2 issues with Deno + Jupyter: running notebooks from cmd and tracking failing tests.



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 18:06**

notebook cmd line issue is described here: [github.com/jupyter/nbcl...](https://github.com/jupyter/nbclient/issues/321). Includes a PR with a suggested fix. In the meantime, using some homemade stuff based on the nbclient fork - [github.com/callmephilip...](https://github.com/callmephilip/jurassic/blob/main/.jurassic/runnb.py)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 18:07**

As far as tests are concerned, ended up dumping them into a separate ts module and then using native `deno test`



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 20:24**

useful reference - PR with Jupyter support for Deno [github.com/denoland/den...](https://github.com/denoland/deno/pull/20337)

